In [1]:
import os
import cv2
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# prepare the dataset
dogsDir = './dataset/Dog/'
catsDir = './dataset/Cat/'

SIZE = (64, 64)

dogs = [dogsDir+i for i in os.listdir(dogsDir)]
cats = [catsDir+i for i in os.listdir(catsDir)]

def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR) # read as grayscale
    return cv2.resize(img, SIZE, interpolation=cv2.INTER_CUBIC) # resize to 64x64

X = []
y = []

def add_to_dataset(arr, label):
    count = len(arr)
    for i, file in enumerate(arr):
        try:
            X.append(read_image(file))
            y.append(label)
            
            if i%250 == 0: print('Processed {} of {}'.format(i, count))
        except:
            pass

add_to_dataset(cats, 0)
add_to_dataset(dogs, 1)

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

Processed 0 of 12494
Processed 250 of 12494
Processed 500 of 12494
Processed 750 of 12494
Processed 1000 of 12494
Processed 1250 of 12494
Processed 1500 of 12494
Processed 1750 of 12494
Processed 2000 of 12494
Processed 2250 of 12494
Processed 2500 of 12494
Processed 2750 of 12494
Processed 3000 of 12494
Processed 3250 of 12494
Processed 3500 of 12494
Processed 3750 of 12494
Processed 4000 of 12494
Processed 4250 of 12494
Processed 4500 of 12494
Processed 4750 of 12494
Processed 5000 of 12494
Processed 5250 of 12494
Processed 5500 of 12494
Processed 5750 of 12494
Processed 6000 of 12494
Processed 6250 of 12494
Processed 6500 of 12494
Processed 6750 of 12494
Processed 7000 of 12494
Processed 7250 of 12494
Processed 7500 of 12494
Processed 7750 of 12494
Processed 8000 of 12494
Processed 8250 of 12494
Processed 8500 of 12494
Processed 8750 of 12494
Processed 9000 of 12494
Processed 9250 of 12494
Processed 9500 of 12494
Processed 9750 of 12494
Processed 10000 of 12494
Processed 10250 of 12

In [3]:
# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [4]:
# train image generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# test image generator
test_datagen = ImageDataGenerator(
    rescale=1./255)

In [5]:
# train batch
batches = train_datagen.flow(X_train, y_train,  batch_size=64)

# validation batch
validation_batches = test_datagen.flow(X_test, y_test,  batch_size=64)

In [6]:
# ** model start **

model = Sequential()

model.add(Convolution2D(32, 5, activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# **model end **

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         204928    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
__________

In [7]:
# compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

In [8]:
# train the model
model.fit_generator(
    batches,
    epochs=80,
    validation_data=validation_batches)

Epoch 1/80
234/234 [==============================] - 73s 310ms/step - loss: 0.6815 - acc: 0.5737 - val_loss: 0.6134 - val_acc: 0.6666
Epoch 2/80
234/234 [==============================] - 67s 287ms/step - loss: 0.6146 - acc: 0.6660 - val_loss: 0.5601 - val_acc: 0.7073
Epoch 3/80
234/234 [==============================] - 67s 287ms/step - loss: 0.5598 - acc: 0.7186 - val_loss: 0.5069 - val_acc: 0.7424
Epoch 4/80
234/234 [==============================] - 67s 286ms/step - loss: 0.5209 - acc: 0.7445 - val_loss: 0.4655 - val_acc: 0.7754
Epoch 5/80
234/234 [==============================] - 67s 287ms/step - loss: 0.4882 - acc: 0.7661 - val_loss: 0.4877 - val_acc: 0.7667
Epoch 6/80
234/234 [==============================] - 67s 287ms/step - loss: 0.4633 - acc: 0.7831 - val_loss: 0.4214 - val_acc: 0.8061
Epoch 7/80
234/234 [==============================] - 67s 287ms/step - loss: 0.4476 - acc: 0.7899 - val_loss: 0.4262 - val_acc: 0.8040
Epoch 8/80
234/234 [==============================] - 6

234/234 [==============================] - 67s 287ms/step - loss: 0.3351 - acc: 0.8645 - val_loss: 0.3256 - val_acc: 0.8733
Epoch 62/80
234/234 [==============================] - 67s 288ms/step - loss: 0.3370 - acc: 0.8633 - val_loss: 0.3186 - val_acc: 0.8614
Epoch 63/80
234/234 [==============================] - 67s 287ms/step - loss: 0.3485 - acc: 0.8555 - val_loss: 0.5063 - val_acc: 0.7809
Epoch 64/80
234/234 [==============================] - 67s 287ms/step - loss: 0.3486 - acc: 0.8557 - val_loss: 0.4012 - val_acc: 0.8208
Epoch 65/80
234/234 [==============================] - 67s 287ms/step - loss: 0.3564 - acc: 0.8596 - val_loss: 0.4084 - val_acc: 0.8496
Epoch 66/80
234/234 [==============================] - 67s 288ms/step - loss: 0.3446 - acc: 0.8562 - val_loss: 0.3211 - val_acc: 0.8660
Epoch 67/80
234/234 [==============================] - 67s 287ms/step - loss: 0.3576 - acc: 0.8537 - val_loss: 0.4191 - val_acc: 0.8094
Epoch 68/80
234/234 [==============================] - 67s 2

In [9]:
# save the model
model.save('dogsvscats.h5')